In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
import random
from sklearn.metrics.pairwise import cosine_similarity

# Visualization
import matplotlib
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
from sklearn.neighbors import NearestNeighbors

#use streamlit for deployment

In [ ]:
fertilizer_df= pd.read_csv("/content/Fertilizer.csv")
crop_df=pd.read_csv("/content/Extended_Crop_Recommendation.csv")
season_df=pd.read_csv("/content/data_season.csv")
yield_df=pd.read_csv("/content/crop_yield.csv")
tunisia_df=pd.read_csv("/content/Tunisie_Sol.csv")

In [ ]:
fertilizer_df.head(5)

,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,37,0,0,Urea
1,12,0,36,DAP
2,7,9,30,Fourteen-Thirty Five-Fourteen
3,22,0,20,Twenty Eight-Twenty Eight
4,35,0,0,Urea


In [ ]:
crop_df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [ ]:
yield_df.head()

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909


In [ ]:
crop_df_crops = crop_df['label'].unique()
yield_df_crops = yield_df['Crop'].unique()

crop_df_crops, season_df_crops, yield_df_crops

(array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
        'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
        'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
        'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee', 'Wheat',
        'Barley', 'green gram', 'peas', 'Horse-gram', 'Groundnut',
        'Sesamum', 'Sunflower', 'Potato', 'Sweet potato', 'Garlic',
        'Onion', 'Coriander', 'Tobacco', 'Urad'], dtype=object),
 array(['Coconut', 'Cocoa', 'Coffee', 'Cardamum', 'Pepper', 'Arecanut',
        'Ginger', 'Tea', 'Paddy', 'Groundnut', 'Blackgram', 'Cashew',
        'Cotton'], dtype=object),
 array(['Arecanut', 'Arhar/Tur', 'Castor seed', 'Coconut ', 'Cotton(lint)',
        'Dry chillies', 'Gram', 'Jute', 'Linseed', 'Maize', 'Mesta',
        'Niger seed', 'Onion', 'Other  Rabi pulses', 'Potato',
        'Rapeseed &Mustard', 'Rice', 'Sesamum', 'Small millets',
        'Sugarcane', 'Sweet potato', 'Tapioca', 'Tobacco', 'Turme

In [ ]:
tunisia_df.head()

,system:index,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,DISP_AREA,EXP1_YEAR,Nitrogen,Phosphorus,Potassium,Profondeur_cm,STATUS,STR1_YEAR,Shape_Area,Shape_Leng,pH,.geo
0,1_1_1_1_1_0000000000000000086a,248,Tunisia,2993,Ariana,NO,3000,73.406243,22.222009,45.421008,0,Member State,1000,0.045516,1.494669,65.196538,"{""type"":""MultiPolygon"",""coordinates"":[[[[10.19..."
1,1_1_1_1_1_0000000000000000086b,248,Tunisia,2994,Ben Arous,NO,3000,77.959755,23.461844,45.623870,0,Member State,1000,0.066773,1.817406,65.518249,"{""type"":""MultiPolygon"",""coordinates"":[[[[10.28..."
2,1_1_1_1_1_0000000000000000086c,248,Tunisia,2995,Bizerte,NO,3000,87.015902,23.211956,45.582688,0,Member State,1000,0.359975,5.609060,63.283688,"{""type"":""MultiPolygon"",""coordinates"":[[[[9.840..."
3,1_1_1_1_1_0000000000000000086d,248,Tunisia,2996,Beja,NO,3000,81.283080,21.718121,45.040040,0,Member State,1000,0.361929,4.071540,64.468094,"{""type"":""MultiPolygon"",""coordinates"":[[[[8.986..."
4,1_1_1_1_1_0000000000000000086e,248,Tunisia,2997,Gabes,NO,3000,25.404086,10.264953,21.478236,0,Member State,1000,0.727515,4.661620,68.484813,"{""type"":""MultiPolygon"",""coordinates"":[[[[10.29..."


Traitement des données

Traitement de la table tunise_sol

In [ ]:
cols_to_drop = ['ADM0_NAME','system:index', 'ADM0_CODE', 'ADM1_CODE', 'DISP_AREA',
                'EXP1_YEAR', 'STATUS', 'STR1_YEAR', '.geo']
tunisia_df_clean = tunisia_df.drop(columns=cols_to_drop)


In [ ]:
tunisia_df_clean = tunisia_df_clean[tunisia_df_clean['Profondeur_cm'].isin([0, 10])]

tunisia_df_clean.head()

,ADM1_NAME,Nitrogen,Phosphorus,Potassium,Profondeur_cm,Shape_Area,Shape_Leng,pH
0,Ariana,73.406243,22.222009,45.421008,0,0.045516,1.494669,65.196538
1,Ben Arous,77.959755,23.461844,45.623870,0,0.066773,1.817406,65.518249
2,Bizerte,87.015902,23.211956,45.582688,0,0.359975,5.609060,63.283688
3,Beja,81.283080,21.718121,45.040040,0,0.361929,4.071540,64.468094
4,Gabes,25.404086,10.264953,21.478236,0,0.727515,4.661620,68.484813


In [ ]:
tunisia_df_clean = tunisia_df_clean.rename(columns={'ADM1_NAME': 'gouvernerat'})
tunisia_df_clean.head()

,gouvernerat,Nitrogen,Phosphorus,Potassium,Profondeur_cm,Shape_Area,Shape_Leng,pH
0,Ariana,73.406243,22.222009,45.421008,0,0.045516,1.494669,65.196538
1,Ben Arous,77.959755,23.461844,45.623870,0,0.066773,1.817406,65.518249
2,Bizerte,87.015902,23.211956,45.582688,0,0.359975,5.609060,63.283688
3,Beja,81.283080,21.718121,45.040040,0,0.361929,4.071540,64.468094
4,Gabes,25.404086,10.264953,21.478236,0,0.727515,4.661620,68.484813


Fusion des fichiers

In [ ]:
# Appliquer la même logique avec le bon nom de colonne

yield_df['Crop_clean'] = yield_df['Crop'].str.lower().str.strip()

# Créer un dictionnaire Crop_clean -> Season
season_dict = yield_df.drop_duplicates(subset='Crop_clean').set_index('Crop_clean')['Season'].to_dict()

# Mapper les saisons sur crop_df
crop_df['label_clean'] = crop_df['label'].str.lower().str.strip()
crop_df['Season'] = crop_df['label_clean'].map(season_dict)

# Supprimer la colonne temporaire
crop_df.drop(columns=['label_clean'], inplace=True)

# Afficher le résultat
crop_df

,N,P,K,temperature,humidity,ph,rainfall,label,Season
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Autumn
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,Autumn
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,Autumn
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,Autumn
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,Autumn
...,...,...,...,...,...,...,...,...,...
2328,39,27,28,27.940000,46.830000,7.140000,563.960000,Urad,Kharif
2329,25,25,28,25.540000,41.600000,6.050000,576.440000,Urad,Kharif
2330,37,22,28,25.890000,42.530000,6.230000,494.140000,Urad,Kharif
2331,41,26,24,29.530000,59.060000,7.200000,460.000000,Urad,Kharif


dictionnaire irrigation

In [ ]:
irrigation_methods = {
    'rice': 'Flood',
    'maize': 'Sprinkler',
    'chickpea': 'Drip',
    'kidneybeans': 'Drip',
    'pigeonpeas': 'Drip',
    'mothbeans': 'Drip',
    'mungbean': 'Drip',
    'blackgram': 'Drip',
    'lentil': 'Drip',
    'pomegranate': 'Drip',
    'banana': 'Sprinkler',
    'mango': 'Drip',
    'grapes': 'Drip',
    'watermelon': 'Sprinkler',
    'muskmelon': 'Sprinkler',
    'apple': 'Sprinkler',
    'orange': 'Drip',
    'papaya': 'Sprinkler',
    'coconut': 'Flood',
    'cotton': 'Drip',
    'jute': 'Flood',
    'coffee': 'Sprinkler',
    'wheat': 'Sprinkler',
    'barley': 'Sprinkler',
    'green gram': 'Drip',
    'peas': 'Drip',
    'horse-gram': 'Drip',
    'groundnut': 'Drip',
    'sesamum': 'Drip',
    'sunflower': 'Drip',
    'potato': 'Sprinkler',
    'sweet potato': 'Sprinkler',
    'garlic': 'Drip',
    'onion': 'Drip',
    'coriander': 'Drip',
    'tobacco': 'Sprinkler',
    'urad': 'Drip'
}

In [ ]:
crop_df['Irrigation'] = crop_df['label'].str.lower().map(irrigation_methods)

In [ ]:
crop_df

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Autumn,Flood
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,Autumn,Flood
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,Autumn,Flood
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,Autumn,Flood
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,Autumn,Flood
...,...,...,...,...,...,...,...,...,...,...
2328,39,27,28,27.940000,46.830000,7.140000,563.960000,Urad,Kharif,Drip
2329,25,25,28,25.540000,41.600000,6.050000,576.440000,Urad,Kharif,Drip
2330,37,22,28,25.890000,42.530000,6.230000,494.140000,Urad,Kharif,Drip
2331,41,26,24,29.530000,59.060000,7.200000,460.000000,Urad,Kharif,Drip


In [ ]:
soil_features = ['Nitrogen', 'Phosphorus', 'Potassium', 'pH']
crop_features = ['N', 'P', 'K', 'ph']

tunisia_values = tunisia_df_clean[soil_features].values
crop_values = crop_df[crop_features].values

# Calcul de la distance euclidienne
distances = euclidean_distances(crop_values, tunisia_values)
closest_indices = np.argmin(distances, axis=1)

# Ajout du gouvernerat à crop_df
crop_df['gouvernerat'] = tunisia_df_clean.iloc[closest_indices]['gouvernerat'].values

In [ ]:
crop_df

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,Autumn,Flood,Bizerte
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice,Autumn,Flood,Bizerte
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice,Autumn,Flood,Tunis
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice,Autumn,Flood,Bizerte
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice,Autumn,Flood,Bizerte
...,...,...,...,...,...,...,...,...,...,...,...
2328,39,27,28,27.940000,46.830000,7.140000,563.960000,Urad,Kharif,Drip,Mahdia
2329,25,25,28,25.540000,41.600000,6.050000,576.440000,Urad,Kharif,Drip,Sfax
2330,37,22,28,25.890000,42.530000,6.230000,494.140000,Urad,Kharif,Drip,Sidi Bouz
2331,41,26,24,29.530000,59.060000,7.200000,460.000000,Urad,Kharif,Drip,Mahdia


Nettoyage des bases de données

In [ ]:
crop_df_crops = crop_df['label'].unique()
crop_df_crops

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee', 'Wheat',
       'Barley', 'green gram', 'peas', 'Horse-gram', 'Groundnut',
       'Sesamum', 'Sunflower', 'Potato', 'Sweet potato', 'Garlic',
       'Onion', 'Coriander', 'Tobacco', 'Urad'], dtype=object)

In [ ]:
tunisia_crops = [
    'wheat', 'barley', 'maize', 'chickpea', 'lentil', 'peas', 'green gram',
    'potato', 'sweet potato', 'garlic', 'onion', 'coriander',
    'orange', 'apple', 'grapes', 'pomegranate', 'watermelon',
    'muskmelon', 'olive', 'citrus', 'mango', 'sunflower', 'groundnut',
    'tobacco',
]

In [ ]:
# Normaliser les cultures pour la comparaison (tout en minuscule)
tunisia_crops_set = set([c.lower() for c in tunisia_crops])
crop_df = crop_df[crop_df['label'].str.lower().isin(tunisia_crops_set)]

In [ ]:
crop_df

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
100,71,54,16,22.613600,63.690706,5.749914,87.759539,maize,Kharif,Sprinkler,Manouba
101,61,44,17,26.100184,71.574769,6.931757,102.266244,maize,Kharif,Sprinkler,Le Kef
102,80,43,16,23.558821,71.593514,6.657965,66.719955,maize,Kharif,Sprinkler,Bizerte
103,73,58,21,19.972160,57.682729,6.596061,60.651715,maize,Kharif,Sprinkler,Bizerte
104,61,38,20,18.478913,62.695039,5.970458,65.438354,maize,Kharif,Sprinkler,Le Kef
...,...,...,...,...,...,...,...,...,...,...,...
2321,49,36,62,22.970000,60.110000,5.790000,529.810000,Tobacco,Whole Year,Sprinkler,Tunis
2322,44,30,68,23.840000,67.780000,5.820000,657.570000,Tobacco,Whole Year,Sprinkler,Ariana
2323,50,44,66,29.580000,63.220000,6.350000,590.190000,Tobacco,Whole Year,Sprinkler,Tunis
2324,43,38,63,26.120000,67.570000,6.420000,524.140000,Tobacco,Whole Year,Sprinkler,Tunis


In [ ]:
# Define the mapping dictionary
season_mapping = {
    "Kharif": "Summer",
    "Rabi": "Winter"
}

# Strip any spaces or hidden characters from 'Season' before replacing
crop_df['Season'] = crop_df['Season'].str.strip().replace(season_mapping)

# Display the first few rows to confirm the changes
crop_df.sample(10)

<ipython-input-283-e454c002e34d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crop_df['Season'] = crop_df['Season'].str.strip().replace(season_mapping)


,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
1550,10,125,196,22.312537,90.035771,5.730557,113.068815,apple,NaN,Sprinkler,Tunis
154,94,54,17,23.391282,61.744272,5.871648,107.319813,maize,Summer,Sprinkler,Bizerte
2211,78,34,48,18.270000,53.820000,6.310000,150.890000,Barley,Whole Year,Sprinkler,Bizerte
1608,39,24,14,30.554726,90.903438,7.189260,106.071198,orange,NaN,Drip,Sidi Bouz
164,76,48,18,19.295634,69.634812,5.775978,83.210306,maize,Summer,Sprinkler,Bizerte
106,93,41,17,25.621717,66.504155,6.047907,105.465470,maize,Summer,Sprinkler,Bizerte
1172,20,30,27,27.810056,51.594455,4.749104,95.898986,mango,NaN,Drip,Sfax
941,5,29,44,21.024329,93.056950,5.578096,104.784701,pomegranate,NaN,Drip,Sfax
1359,81,6,55,24.889105,85.870591,6.110143,51.706991,watermelon,NaN,Sprinkler,Bizerte
1622,8,7,10,28.262049,91.983174,6.929216,105.213226,orange,NaN,Drip,Tataouine


In [ ]:
null_season_df = crop_df[crop_df['Season'].isna()]
null_season_df_crops = null_season_df['label'].unique()
null_season_df_crops

array(['chickpea', 'lentil', 'pomegranate', 'mango', 'grapes',
       'watermelon', 'muskmelon', 'apple', 'orange', 'green gram', 'peas'],
      dtype=object)

In [ ]:
season_dict = {
    'chickpea': 'Winter',         # grows mainly through the winter
    'lentil': 'Winter',           # develops during winter
    'pomegranate': 'Summer',      # matures in late summer
    'mango': 'Summer',            # ripens in summer
    'grapes': 'Summer',           # grows and ripens in summer
    'watermelon': 'Summer',       # rapid growth in early summer
    'muskmelon': 'Spring',        # grows mainly in spring
    'apple': 'Autumn',            # matures mostly in autumn
    'orange': 'Winter',           # develops and harvested in winter
    'green gram': 'Summer',       # summer short-cycle pulse
    'peas': 'Winter'              # slow development through winter
}

In [ ]:
crop_df.loc[crop_df['Season'].isna(), 'Season'] = crop_df.loc[crop_df['Season'].isna(), 'label'].str.lower().map(season_dict)

In [ ]:
null_season_df = crop_df[crop_df['Season'].isna()]
null_season_df_crops = null_season_df['label'].unique()
null_season_df_crops

array([], dtype=object)

In [ ]:
crop_df

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
100,71,54,16,22.613600,63.690706,5.749914,87.759539,maize,Summer,Sprinkler,Manouba
101,61,44,17,26.100184,71.574769,6.931757,102.266244,maize,Summer,Sprinkler,Le Kef
102,80,43,16,23.558821,71.593514,6.657965,66.719955,maize,Summer,Sprinkler,Bizerte
103,73,58,21,19.972160,57.682729,6.596061,60.651715,maize,Summer,Sprinkler,Bizerte
104,61,38,20,18.478913,62.695039,5.970458,65.438354,maize,Summer,Sprinkler,Le Kef
...,...,...,...,...,...,...,...,...,...,...,...
2321,49,36,62,22.970000,60.110000,5.790000,529.810000,Tobacco,Whole Year,Sprinkler,Tunis
2322,44,30,68,23.840000,67.780000,5.820000,657.570000,Tobacco,Whole Year,Sprinkler,Ariana
2323,50,44,66,29.580000,63.220000,6.350000,590.190000,Tobacco,Whole Year,Sprinkler,Tunis
2324,43,38,63,26.120000,67.570000,6.420000,524.140000,Tobacco,Whole Year,Sprinkler,Tunis


In [ ]:
crop_df['Season'].value_counts(ascending=False)

,count
Season,
Summer,514
Winter,328
Spring,100
Autumn,100
Whole Year,56


In [ ]:
print("Duplicate Values =",crop_df.duplicated().sum()) #duplicate

Duplicate Values = 0


encodage des colonnes

In [ ]:
# Remplacement direct des colonnes par les valeurs encodées

# Encodage simple et logique des saisons
season_order = {
    'Winter': 1,
    'Spring': 2,
    'Summer': 3,
    'Autumn': 4,
    'Whole Year': 5
}
crop_df['Season'] = crop_df['Season'].map(season_order)

# Encodage des cultures
unique_crops = sorted(crop_df['label'].str.lower().unique())
crop_encoding = {crop: idx+1 for idx, crop in enumerate(unique_crops)}
crop_df['label'] = crop_df['label'].str.lower().map(crop_encoding)

# Encodage des gouvernerats
unique_govs = sorted(crop_df['gouvernerat'].dropna().unique())
gov_encoding = {gov: idx+1 for idx, gov in enumerate(unique_govs)}
crop_df['gouvernerat'] = crop_df['gouvernerat'].map(gov_encoding)

# Encodage de l'irrigation (corrigé)
unique_irrigations = sorted(crop_df['Irrigation'].dropna().unique())
irrigation_encoding = {irrig: idx + 1 for idx, irrig in enumerate(unique_irrigations)}
crop_df['Irrigation'] = crop_df['Irrigation'].map(irrigation_encoding)

crop_df


<ipython-input-291-42b9cd31a70a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crop_df['Season'] = crop_df['Season'].map(season_order)
<ipython-input-291-42b9cd31a70a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crop_df['label'] = crop_df['label'].str.lower().map(crop_encoding)
<ipython-input-291-42b9cd31a70a>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
100,71,54,16,22.613600,63.690706,5.749914,87.759539,10,3,2,9
101,61,44,17,26.100184,71.574769,6.931757,102.266244,10,3,2,7
102,80,43,16,23.558821,71.593514,6.657965,66.719955,10,3,2,2
103,73,58,21,19.972160,57.682729,6.596061,60.651715,10,3,2,2
104,61,38,20,18.478913,62.695039,5.970458,65.438354,10,3,2,7
...,...,...,...,...,...,...,...,...,...,...,...
2321,49,36,62,22.970000,60.110000,5.790000,529.810000,20,5,2,16
2322,44,30,68,23.840000,67.780000,5.820000,657.570000,20,5,2,1
2323,50,44,66,29.580000,63.220000,6.350000,590.190000,20,5,2,16
2324,43,38,63,26.120000,67.570000,6.420000,524.140000,20,5,2,16


In [ ]:
print("valeur null \n",crop_df.isna().sum()) #duplicate

valeur null 
 N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
Season         0
Irrigation     0
gouvernerat    0
dtype: int64


Modélisation

In [ ]:
features = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall',
            'label', 'Season', 'Irrigation', 'gouvernerat']

X = crop_df[features].values

In [ ]:
# --- ENCODAGE DES ENTRÉES UTILISATEUR ---
def encode_user_input(user_input):
    return [
        user_input['N'],
        user_input['P'],
        user_input['K'],
        user_input['temperature'],
        user_input['humidity'],
        user_input['ph'],
        user_input['rainfall'],
        crop_encoding.get(user_input['label'].lower(), 0),
        season_order.get(user_input['Season'], 0),
        gov_encoding.get(user_input['gouvernerat'], 0),
        irrigation_encoding.get(user_input['Irrigation'], 0)
    ]

In [ ]:
# --- DÉCODAGE POUR AFFICHER DES RÉSULTATS LISIBLES ---
# Dictionnaires inverses pour retrouver les noms à partir des codes
crop_decoding = {v: k for k, v in crop_encoding.items()}
season_decoding = {v: k for k, v in season_order.items()}
gov_decoding = {v: k for k, v in gov_encoding.items()}
irrigation_decoding = {v: k for k, v in irrigation_encoding.items()}

def decode_results(df):
    df = df.copy()
    df['label'] = df['label'].map(crop_decoding)
    df['Season'] = df['Season'].map(season_decoding)
    df['gouvernerat'] = df['gouvernerat'].map(gov_decoding)
    df['Irrigation'] = df['Irrigation'].map(irrigation_decoding)
    return df



In [ ]:
# --- SYSTÈME DE RECOMMANDATION ---
def recommend_crops(input_vector, top_n=5):
    features = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall',
                'label', 'Season', 'gouvernerat', 'Irrigation']
    X = crop_df[features].values
    similarities = cosine_similarity([input_vector], X)
    top_indices = np.argsort(similarities[0])[::-1][:top_n]
    top_df = crop_df.iloc[top_indices]
    return decode_results(top_df)

In [ ]:
user_input = {
    'N': 70,
    'P': 30,
    'K': 45,
    'temperature': 24,
    'humidity': 70,
    'ph': 6.4,
    'rainfall': 180,
    'label': 'chickpea',
    'Season': 'Spring',
    'gouvernerat': 'Sfax',
    'Irrigation': 'Drip'
}

input_vector = encode_user_input(user_input)
recommend_crops(input_vector, top_n=5)


,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
2207,64,33,48,16.930000,51.920000,6.530000,166.090000,barley,Whole Year,Sprinkler,Ariana
2211,78,34,48,18.270000,53.820000,6.310000,150.890000,barley,Whole Year,Sprinkler,Bizerte
2208,73,39,32,16.620000,45.510000,6.320000,152.170000,barley,Whole Year,Sprinkler,Bizerte
1124,38,15,30,28.918620,48.139745,5.075505,97.013316,mango,Summer,Drip,Sidi Bouz
1121,36,25,33,27.983928,53.330189,5.548585,99.614657,mango,Summer,Drip,Mahdia


système de recommandation cas par cas

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity_recommendations(input_vector, features, top_n=5):
    X = crop_df[features].values
    similarities = cosine_similarity([input_vector], X)
    top_indices = similarities[0].argsort()[::-1][:top_n]
    return decode_results(crop_df.iloc[top_indices])

Recommandation par crop


In [ ]:
def recommend_by_crop_balanced(crop_name, top_n=5):
    # Extraire la ligne correspondant à la crop d'origine
    crop_row = crop_df[crop_df['label'] == crop_encoding.get(crop_name.lower())]
    if crop_row.empty:
        return f"❌ Crop '{crop_name}' not found."

    # Définir les features à comparer
    features = ['N', 'P', 'K', 'ph', 'temperature', 'humidity', 'rainfall', 'Season', 'Irrigation']

    # Vecteur d'entrée
    input_vector = crop_row.iloc[0][features].values

    # Calcul des similarités
    X = crop_df[features].values
    similarities = cosine_similarity([input_vector], X)

    # Obtenir les indices triés par similarité
    top_indices = np.argsort(similarities[0])[::-1]

    # Préparer les résultats avec une seule ligne pour la crop d’origine
    seen_labels = set()
    results = []

    for idx in top_indices:
        row = crop_df.iloc[idx]
        decoded_label = [k for k, v in crop_encoding.items() if v == row['label']][0]
        if decoded_label == crop_name.lower() and crop_name.lower() not in seen_labels:
            results.append(row)
            seen_labels.add(decoded_label)
        elif decoded_label != crop_name.lower() and decoded_label not in seen_labels:
            results.append(row)
            seen_labels.add(decoded_label)
        if len(results) >= top_n:
            break

    # Convertir en DataFrame et décoder les valeurs
    df_result = pd.DataFrame(results)
    df_result['label'] = df_result['label'].map({v: k for k, v in crop_encoding.items()})
    df_result['Season'] = df_result['Season'].map({v: k for k, v in season_order.items()})
    df_result['Irrigation'] = df_result['Irrigation'].map({v: k for k, v in irrigation_encoding.items()})
    df_result['gouvernerat'] = df_result['gouvernerat'].map({v: k for k, v in gov_encoding.items()})

    return df_result.reset_index(drop=True)

Recommandation par saison

In [ ]:
def recommend_by_season_balanced(season, top_n=5):
    season_code = season_order.get(season, 0)
    if season_code == 0:
        return f"❌ Season '{season}' not recognized."

    # 1️⃣ Étape 1 : récupérer au moins une culture de la saison demandée
    seasonal_crops = crop_df[crop_df['Season'] == season_code]
    seen_labels = set()
    results = []

    if not seasonal_crops.empty:
        # Ajouter la première culture de cette saison
        for _, row in seasonal_crops.iterrows():
            if row['label'] not in seen_labels:
                results.append(row)
                seen_labels.add(row['label'])
                break  # on veut UNE seule ligne de la saison demandée

    # 2️⃣ Étape 2 : chercher des cultures proches de la saison demandée
    # Calculer le profil moyen de la saison demandée
    if seasonal_crops.empty:
        return "❌ No crop registered for this season."

    mean_vector = seasonal_crops[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].mean().values
    similarities = cosine_similarity([mean_vector], crop_df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].values)
    top_indices = np.argsort(similarities[0])[::-1]

    # 3️⃣ Étape 3 : compléter avec des cultures variées, autres que celle ajoutée
    for idx in top_indices:
        row = crop_df.iloc[idx]
        if row['label'] not in seen_labels:
            results.append(row)
            seen_labels.add(row['label'])
        if len(results) >= top_n:
            break

    # 4️⃣ Décodage
    df_result = pd.DataFrame(results)
    df_result['label'] = df_result['label'].map({v: k for k, v in crop_encoding.items()})
    df_result['Season'] = df_result['Season'].map({v: k for k, v in season_order.items()})
    df_result['Irrigation'] = df_result['Irrigation'].map({v: k for k, v in irrigation_encoding.items()})
    df_result['gouvernerat'] = df_result['gouvernerat'].map({v: k for k, v in gov_encoding.items()})

    return df_result.reset_index(drop=True)


Recommandation par irrigation


In [ ]:
def recommend_by_irrigation_balanced(irrigation_type, top_n=5):
    # Récupération du code d'irrigation
    irrigation_code = irrigation_encoding.get(irrigation_type, 0)
    if irrigation_code == 0:
        return f"❌ Irrigation type '{irrigation_type}' not found."

    # Filtrer les cultures correspondant à cette méthode d'irrigation
    filtered_crops = crop_df[crop_df['Irrigation'] == irrigation_code]
    if filtered_crops.empty:
        return f"❌ No crops found for irrigation '{irrigation_type}'."

    # Calcul de la moyenne des conditions
    mean_vector = filtered_crops[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].mean().values

    # Calcul de la similarité par rapport au profil moyen
    features = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
    X = crop_df[features].values
    similarities = cosine_similarity([mean_vector], X)
    top_indices = np.argsort(similarities[0])[::-1]

    # Construction des suggestions diversifiées
    seen_labels = set()
    results = []
    for idx in top_indices:
        row = crop_df.iloc[idx]
        if row['Irrigation'] == irrigation_code and row['label'] not in seen_labels:
            results.append(row)
            seen_labels.add(row['label'])
        if len(results) >= top_n:
            break

    # Décodage
    df_result = pd.DataFrame(results)
    df_result['label'] = df_result['label'].map({v: k for k, v in crop_encoding.items()})
    df_result['Season'] = df_result['Season'].map({v: k for k, v in season_order.items()})
    df_result['Irrigation'] = df_result['Irrigation'].map({v: k for k, v in irrigation_encoding.items()})
    df_result['gouvernerat'] = df_result['gouvernerat'].map({v: k for k, v in gov_encoding.items()})

    return df_result.reset_index(drop=True)


Recommandation intelligente mixte (cas par cas)

In [ ]:
def recommend_case_by_case_balanced(user_input, top_n=5):
    vectors = []

    if 'label' in user_input:
        label_code = crop_encoding.get(user_input['label'].lower(), None)
        crop_row = crop_df[crop_df['label'] == label_code]
        if not crop_row.empty:
            vectors.append(crop_row.iloc[0][['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].values)

    if 'Season' in user_input:
        season_code = season_order.get(user_input['Season'], 0)
        season_rows = crop_df[crop_df['Season'] == season_code]
        if not season_rows.empty:
            vectors.append(season_rows[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].mean().values)

    if 'Irrigation' in user_input:
        irrig_code = irrigation_encoding.get(user_input['Irrigation'], 0)
        irrig_rows = crop_df[crop_df['Irrigation'] == irrig_code]
        if not irrig_rows.empty:
            vectors.append(irrig_rows[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].mean().values)

    if not vectors:
        return "❌ No valid input provided."

    # Moyenne des profils combinés
    final_vector = np.mean(vectors, axis=0)

    # Similarité
    features = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
    X = crop_df[features].values
    similarities = cosine_similarity([final_vector], X)
    top_indices = np.argsort(similarities[0])[::-1]

    # Suggestions variées
    seen_labels = set()
    results = []
    for idx in top_indices:
        row = crop_df.iloc[idx]
        if row['label'] not in seen_labels:
            results.append(row)
            seen_labels.add(row['label'])
        if len(results) >= top_n:
            break

    # Décodage
    df_result = pd.DataFrame(results)
    df_result['label'] = df_result['label'].map({v: k for k, v in crop_encoding.items()})
    df_result['Season'] = df_result['Season'].map({v: k for k, v in season_order.items()})
    df_result['Irrigation'] = df_result['Irrigation'].map({v: k for k, v in irrigation_encoding.items()})
    df_result['gouvernerat'] = df_result['gouvernerat'].map({v: k for k, v in gov_encoding.items()})

    return df_result.reset_index(drop=True)


In [ ]:
recommend_by_crop_balanced('chickpea', top_n=5)

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
0,40.0,72.0,77.0,17.024985,16.988612,7.485996,88.551231,chickpea,Winter,Drip,Tunis
1,35.0,142.0,203.0,21.170892,90.237302,5.895319,123.649515,apple,Autumn,Sprinkler,Tunis
2,27.0,47.0,35.0,18.390000,44.920000,7.260000,63.540000,lentil,Winter,Drip,Sidi Bouz
3,62.0,45.0,34.0,15.440000,45.170000,6.020000,95.130000,barley,Whole Year,Sprinkler,Ariana
4,39.0,132.0,196.0,35.830891,83.325601,5.778594,73.679849,grapes,Summer,Drip,Tunis


In [ ]:
recommend_by_season_balanced('Spring', top_n=5)

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
0,115.0,17.0,55.0,27.578269,94.118782,6.776533,28.082532,muskmelon,Spring,Sprinkler,Jendouba
1,95.0,16.0,55.0,25.269312,87.550551,6.612848,40.126504,watermelon,Summer,Sprinkler,Jendouba
2,99.0,38.0,21.0,22.883309,71.597224,6.352472,67.727773,maize,Summer,Sprinkler,Jendouba
3,110.0,57.0,42.0,15.100000,57.310000,6.370000,106.600000,wheat,Winter,Sprinkler,Jendouba
4,40.0,61.0,22.0,20.949818,65.810876,7.002216,44.239130,lentil,Winter,Drip,Kairouan


In [ ]:
recommend_by_irrigation_balanced('Drip', top_n=5)

,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
0,23.0,45.0,36.0,24.360000,48.340000,6.300000,94.970000,lentil,Winter,Drip,Sfax
1,22.0,54.0,38.0,21.120000,46.770000,8.850000,98.390000,chickpea,Winter,Drip,Sfax
2,24.0,33.0,35.0,29.263829,54.822579,5.342866,100.758623,mango,Summer,Drip,Sfax
3,23.0,30.0,44.0,20.938929,85.429129,6.124761,103.029594,pomegranate,Summer,Drip,Sfax
4,22.0,30.0,12.0,15.781442,92.510777,6.354007,119.035002,orange,Winter,Drip,Gabes


In [ ]:
recommend_case_by_case_balanced({'label': 'chickpea', 'Season': 'Autumn'})


,N,P,K,temperature,humidity,ph,rainfall,label,Season,Irrigation,gouvernerat
0,35.0,142.0,203.0,21.170892,90.237302,5.895319,123.649515,apple,Autumn,Sprinkler,Tunis
1,28.0,72.0,84.0,18.729631,19.181973,6.481783,71.580102,chickpea,Winter,Drip,Tunis
2,39.0,132.0,196.0,35.830891,83.325601,5.778594,73.679849,grapes,Summer,Drip,Tunis
3,27.0,47.0,35.0,18.390000,44.920000,7.260000,63.540000,lentil,Winter,Drip,Sidi Bouz
4,8.0,38.0,32.0,29.751508,46.737233,4.981817,91.405983,mango,Summer,Drip,Sfax


Enrichissement de la base donnée

In [10]:
#import requests
#import pandas as pd

# Titres exacts pour chaque culture
# correct_titles = {
#     'wheat': 'Wheat',
#     'barley': 'Barley',
#     'peas': 'Pisum_sativum',
#     'garlic': 'Garlic',
#     'onion': 'Onion',
#     'orange': 'Orange_(fruit)',
#     'grapes': 'Grape',
#     'muskmelon': 'Cucumis_melo'
# }

# wiki_api_base = "https://en.wikipedia.org/api/rest_v1/page/summary/"

# results = []

# for crop, title in correct_titles.items():
#     url = wiki_api_base + title
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             data = response.json()
#             desc = data.get("extract", "No description found")
#             results.append({"Crop": crop, "WikiTitle": title.replace("_", " "), "Description": desc})
#             print(f"✅ {crop}")
#         else:
#             results.append({"Crop": crop, "WikiTitle": title, "Description": f"HTTP error {response.status_code}"})
#     except Exception as e:
#         results.append({"Crop": crop, "WikiTitle": title, "Description": str(e)})

# df = pd.DataFrame(results)
# df.to_csv("fixed_wikipedia_crop_descriptions.csv", index=False)
# print("✅ Fichier généré : fixed_wikipedia_crop_descriptions.csv")
# print(df.head())


✅ wheat
✅ barley
✅ peas
✅ garlic
✅ onion
✅ orange
✅ grapes
✅ muskmelon
✅ Fichier généré : fixed_wikipedia_crop_descriptions.csv
     Crop      WikiTitle                                        Description
0   wheat          Wheat  Wheat is a group of wild and domesticated gras...
1  barley         Barley  Barley, a member of the grass family, is a maj...
2    peas  Pisum sativum  Pea is a pulse, vegetable or fodder crop, but ...
3  garlic         Garlic  Garlic is a species of bulbous flowering plant...
4   onion          Onion  An onion, also known as the bulb onion or comm...


In [21]:
# crops_description= pd.read_csv("/content/tunisia_crops_descriptions_1.csv")
# crops_description.tail()

,Crop,Description
19,garlic,Garlic is a species of bulbous flowering plant...
20,onion,"An onion, also known as the bulb onion or comm..."
21,orange,"The orange, also called sweet orange to distin..."
22,grapes,"A grape is a fruit, botanically a berry, of th..."
23,muskmelon,"Cucumis melo, also known as melon, is a specie..."


In [16]:
# Charger le fichier CSV
# df = pd.read_csv("/content/tunisia_crops_descriptions_1.csv")

# Liste des cultures à supprimer
# crops_to_remove = ['wheat', 'barley', 'peas', 'garlic', 'onion', 'orange', 'grapes', 'muskmelon']

# Supprimer les lignes où la colonne 'Crop' contient ces cultures
# df_filtered = df[~df['Crop'].isin(crops_to_remove)]

# Sauvegarder dans un nouveau fichier (ou écraser l'ancien si tu veux)
# df_filtered.to_csv("/content/tunisia_crops_descriptions_1.csv", index=False)


In [19]:
# Charger le fichier d'origine et celui avec les nouvelles descriptions
# original_path = "/content/tunisia_crops_descriptions_1.csv"
# new_data_path = "/content/fixed_wikipedia_crop_descriptions.csv"

# df_original = pd.read_csv(original_path)
# df_new = pd.read_csv(new_data_path)

# Supprimer la colonne WikiTitle du nouveau fichier
# if 'WikiTitle' in df_new.columns:
#     df_new = df_new.drop(columns=['WikiTitle'])

# Supprimer les lignes spécifiques de l'ancien fichier
# crops_to_remove = ['wheat', 'barley', 'peas', 'garlic', 'onion', 'orange', 'grapes', 'muskmelon']
# df_original_filtered = df_original[~df_original['Crop'].isin(crops_to_remove)]

# Concaténer les deux
# df_final = pd.concat([df_original_filtered, df_new], ignore_index=True)

# Sauvegarder dans le même fichier
# output_path = "/content/tunisia_crops_descriptions_1.csv"
# df_final.to_csv(output_path, index=False)

In [24]:
# import requests

# Cultures mal décrites et leurs titres corrigés pour Wikipedia
# corrections = {
#     'maize': 'Maize',
#     'chickpea': 'Chickpea',
#     'lentil': 'Lentil',
#     'coriander': 'Coriander',
#     'apple': 'Apple'
# }

# Utilisation de l'API REST de Wikipedia
# wiki_api_base = "https://en.wikipedia.org/api/rest_v1/page/summary/"

# results = []

# for crop, title in corrections.items():
#     url = wiki_api_base + title.replace(" ", "_")
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             data = response.json()
#             desc = data.get("extract", "No description found")
#             results.append({"Crop": crop, "Description": desc})
#         else:
#             results.append({"Crop": crop, "Description": f"HTTP error {response.status_code}"})
#     except Exception as e:
#         results.append({"Crop": crop, "Description": str(e)})

# # Résultats dans un DataFrame
# df_fixed = pd.DataFrame(results)
# df_fixed.head()
# output_path = "/content/tunisia_crops_descriptions_1.csv"
# df_fixed.to_csv(output_path, index=False)


modelisation llm

Déploiement